In [54]:
import gymnasium as gym
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random
from collections import deque

# ==========================================
# 1. Q-Network & 2. ReplayBuffer (이전과 동일)
# ==========================================
class QNetwork(nn.Module):
    def __init__(self, state_size, action_size):
        super(QNetwork, self).__init__()
        self.fc1 = nn.Linear(state_size, 24)
        self.fc2 = nn.Linear(24, 24)
        self.fc3 = nn.Linear(24, action_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

class ReplayBuffer:
    def __init__(self, capacity=10000):
        self.memory = deque(maxlen=capacity)

    def add(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def size(self):
        return len(self.memory)

# ==========================================
# 3. DQN 학습 메인 루프 (수정됨)
# ==========================================
def train_dqn():
    MAX_SCORE = 50000

    env = gym.make('CartPole-v1', max_episode_steps=MAX_SCORE)
    state_size = env.observation_space.shape[0]
    action_size = env.action_space.n
    env.unwrapped.x_threshold = 10.0
    env.unwrapped.theta_threshold_radians = 90 * np.pi / 180

    # 설정이 잘 되었는지 확인
    print(f"변경된 최대 스텝 수: {env.spec.max_episode_steps}")
    
    policy_net = QNetwork(state_size, action_size)
    target_net = QNetwork(state_size, action_size)
    target_net.load_state_dict(policy_net.state_dict()) 

    optimizer = optim.Adam(policy_net.parameters(), lr=0.005)
    memory = ReplayBuffer(capacity=100000)

    # 🛠️ [수정 1] 하이퍼파라미터 복구 및 스텝 업데이트 설정
    batch_size = 128
    gamma = 0.99 
    epsilon = 1.0 
    epsilon_decay = 0.9995 # 9를 다시 하나 추가했습니다! (천천히 탐험 감소)
    epsilon_min = 0.01
    
    target_update_steps = 5000 # 1000 스텝(프레임)마다 타겟 업데이트
    total_steps = 0            # 누적 스텝을 세기 위한 변수 추가

    num_episodes = 20000 
    best_score = 0 

    TAU = 0.005

    print("🚀 학습을 시작합니다! (DDQN + Step Update 버전)")
    print(f"{"에피소드":<10} | {"점수":<10} | {"Epsilon":<10} | {"누적 스텝":<10}")

    for episode in range(num_episodes):
        state, _ = env.reset()
        total_reward = 0
        done = False
        truncated = False

        while not (done or truncated):
            total_steps += 1 # 스텝 1 증가

            # [행동 선택]
            if random.random() <= epsilon:
                action = env.action_space.sample() 
            else:
                state_tensor = torch.FloatTensor(state).unsqueeze(0)
                with torch.no_grad():
                    q_values = policy_net(state_tensor)
                action = np.argmax(q_values.numpy()) 

            # [환경과 상호작용]
            next_state, reward, done, truncated, _ = env.step(action)
            total_reward += reward

            if done:
                reward = -10 

            # [메모리 저장]
            memory.add(state, action, reward, next_state, done)
            state = next_state

            # [학습]
            if memory.size() > batch_size:
                batch = memory.sample(batch_size)
                
                states, actions, rewards, next_states, dones = zip(*batch)
                states = torch.FloatTensor(np.array(states))
                actions = torch.LongTensor(actions).unsqueeze(1)
                rewards = torch.FloatTensor(rewards).unsqueeze(1)
                next_states = torch.FloatTensor(np.array(next_states))
                dones = torch.FloatTensor(dones).unsqueeze(1)

                current_q = policy_net(states).gather(1, actions)
                
                # 🛠️ [수정 3] DDQN 코드에 .detach() 추가!
                best_actions = policy_net(next_states).argmax(1).unsqueeze(1)
                # target_net의 결과값은 역전파를 하지 않으므로 detach()로 끊어줍니다.
                max_next_q = target_net(next_states).gather(1, best_actions).detach() 

                target_q = rewards + (gamma * max_next_q * (1 - dones))

                loss = nn.MSELoss()(current_q, target_q)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            for target_param, policy_param in zip(target_net.parameters(), policy_net.parameters()):
                target_param.data.copy_(TAU * policy_param.data + (1.0 - TAU) * target_param.data)

        # 에피소드 종료 후 Epsilon 감소
        if epsilon > epsilon_min:
            epsilon *= epsilon_decay 

        if (episode + 1) % 10 == 0:
            print(f"{f"{episode+1}/{num_episodes}":<11}| {total_reward:<10} | Epsilon: {epsilon:.2f} | {total_steps:<10}")

        # 신기록 저장 및 조기 종료
        if total_reward > best_score:
            best_score = total_reward
            torch.save(policy_net.state_dict(), 'best_dqn_model.pth')
            print(f"🌟 신기록 달성! [{best_score}점] - 모델 저장 완료!")
            
            if best_score >= MAX_SCORE:
                print(f"🏆 목표 점수({MAX_SCORE}점) 도달! 학습을 조기 종료합니다.")
                break
                
    print("🎉 학습 완료!")
    env.close()

if __name__ == "__main__":
    train_dqn()

변경된 최대 스텝 수: 50000
🚀 학습을 시작합니다! (DDQN + Step Update 버전)
🌟 신기록 달성! [26.0점] - 모델 저장 완료!
🌟 신기록 달성! [44.0점] - 모델 저장 완료!
에피소드: 10/20000 | 점수: 10.0 | Epsilon: 1.00 | 누적 스텝: 188
🌟 신기록 달성! [55.0점] - 모델 저장 완료!
에피소드: 20/20000 | 점수: 19.0 | Epsilon: 0.99 | 누적 스텝: 390
🌟 신기록 달성! [63.0점] - 모델 저장 완료!
에피소드: 30/20000 | 점수: 23.0 | Epsilon: 0.99 | 누적 스텝: 667
에피소드: 40/20000 | 점수: 26.0 | Epsilon: 0.98 | 누적 스텝: 905
에피소드: 50/20000 | 점수: 31.0 | Epsilon: 0.98 | 누적 스텝: 1196
에피소드: 60/20000 | 점수: 20.0 | Epsilon: 0.97 | 누적 스텝: 1391
에피소드: 70/20000 | 점수: 30.0 | Epsilon: 0.97 | 누적 스텝: 1611
에피소드: 80/20000 | 점수: 38.0 | Epsilon: 0.96 | 누적 스텝: 1903
에피소드: 90/20000 | 점수: 14.0 | Epsilon: 0.96 | 누적 스텝: 2107
에피소드: 100/20000 | 점수: 55.0 | Epsilon: 0.95 | 누적 스텝: 2382
에피소드: 110/20000 | 점수: 36.0 | Epsilon: 0.95 | 누적 스텝: 2624
에피소드: 120/20000 | 점수: 23.0 | Epsilon: 0.94 | 누적 스텝: 2904
에피소드: 130/20000 | 점수: 22.0 | Epsilon: 0.94 | 누적 스텝: 3112
에피소드: 140/20000 | 점수: 12.0 | Epsilon: 0.93 | 누적 스텝: 3333
에피소드: 150/20000 | 점수: 17.0 | Epsilon: 0

KeyboardInterrupt: 

In [57]:
import gymnasium as gym
import torch
import torch.nn as nn
import numpy as np
import time

# ==========================================
# 1. 뇌 구조(Q-Network) 정의
# (저장된 파일을 읽어오려면 뼈대가 먼저 있어야 합니다)
# ==========================================
class QNetwork(nn.Module):
    def __init__(self, state_size, action_size):
        super(QNetwork, self).__init__()
        self.fc1 = nn.Linear(state_size, 24)
        self.fc2 = nn.Linear(24, 24)
        self.fc3 = nn.Linear(24, action_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

# ==========================================
# 2. 훈련된 '뇌' 불러오기 및 실전 모드 설정
# ==========================================
trained_model = QNetwork(state_size=4, action_size=2)
trained_model.load_state_dict(torch.load('best_dqn_model.pth', weights_only=True))
trained_model.eval() # 실전(평가) 모드 켬

# ==========================================
# 3. 실전 행동 결정 함수
# ==========================================
def act_in_real_world(state, model):
    with torch.no_grad(): # 기울기 계산 끔 (속도 향상)
        # 신경망에 넣기 위해 리스트 형태의 state를 텐서로 변환
        state_tensor = torch.FloatTensor(state).unsqueeze(0) 
        q_values = model(state_tensor)
        
        # 무작위 행동 없이 Q값이 가장 높은 최적의 행동 100% 선택
        best_action = np.argmax(q_values.numpy())
        return best_action

# ==========================================
# 4. 실제 게임 화면 띄우기 (시각화 루프)
# ==========================================
# render_mode="human" 옵션이 팝업창으로 게임 화면을 보여줍니다!
env = gym.make('CartPole-v1', render_mode="human", max_episode_steps=500)

print("📺 훈련된 AI의 플레이를 감상하세요!")

# 총 3판의 게임을 보여줍니다.
for episode in range(3):
    state, _ = env.reset()
    done = False
    truncated = False
    score = 0
    
    print(f"\n--- {episode + 1}번째 게임 시작! ---")
    
    while not (done or truncated):
        # (1) AI에게 현재 상태를 보여주고 행동을 결정받습니다.
        action = act_in_real_world(state, trained_model)
        
        # (2) 결정된 행동을 게임 환경에 실행합니다.
        state, reward, done, truncated, _ = env.step(action)
        score += reward
        
    print(f"게임 종료! 최종 점수(버틴 시간): {score} 프레임")

env.close()

📺 훈련된 AI의 플레이를 감상하세요!

--- 1번째 게임 시작! ---
게임 종료! 최종 점수(버틴 시간): 500.0 프레임

--- 2번째 게임 시작! ---
게임 종료! 최종 점수(버틴 시간): 500.0 프레임

--- 3번째 게임 시작! ---
게임 종료! 최종 점수(버틴 시간): 500.0 프레임


In [61]:
import gymnasium as gym
import torch
import torch.nn as nn
import numpy as np
import random
import time

# ==========================================
# 1. 뇌 구조 정의 (이전과 완벽히 동일)
# ==========================================
class QNetwork(nn.Module):
    def __init__(self, state_size, action_size):
        super(QNetwork, self).__init__()
        self.fc1 = nn.Linear(state_size, 24)
        self.fc2 = nn.Linear(24, 24)
        self.fc3 = nn.Linear(24, action_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

# ==========================================
# 2. 만점짜리 뇌 불러오기
# ==========================================
trained_model = QNetwork(state_size=4, action_size=2)
# 저장해둔 신기록 모델을 불러옵니다.
trained_model.load_state_dict(torch.load('best_dqn_model.pth', weights_only=True))
trained_model.eval() # 실전 모드 ON!

def act_in_real_world(state, model):
    with torch.no_grad():
        state_tensor = torch.FloatTensor(state).unsqueeze(0)
        q_values = model(state_tensor)
        return np.argmax(q_values.numpy())

# ==========================================
# 3. 지옥의 링(환경) 띄우기
# ==========================================
env = gym.make('CartPole-v1', render_mode="human", max_episode_steps=1000)

print("😈 극한의 방해꾼 테스트를 시작합니다! 화면을 주목하세요!")

for episode in range(5): # 총 5판 진행
    state, _ = env.reset()
    done = False
    truncated = False
    score = 0
    
    print(f"\n--- {episode + 1}번째 가혹한 테스트 시작! ---")
    
    while not (done or truncated):
        # AI의 완벽한 방어
        action = act_in_real_world(state, trained_model)
        state, reward, done, truncated, _ = env.step(action)
        
        # ========================================================
        # 💥 [방해꾼 로직] 5% 확률로 냅다 카트 걷어차기!
        # ========================================================
        if random.random() < 0.01:
            current_physics = list(env.unwrapped.state)
            
            # 카트의 '속도'에 강제로 -2.0 ~ 2.0 사이의 엄청난 힘을 가합니다.
            push_force = random.uniform(-2.0, 2.0)
            current_physics[1] += push_force
            
            env.unwrapped.state = np.array(current_physics)
            
            direction = "오른쪽 ➡️" if push_force > 0 else "⬅️ 왼쪽"
            print(f"💥 퍽! {direction}으로 걷어찼습니다! (가해진 힘: {push_force:.2f})")
        # ========================================================

        score += reward
        
    print(f"테스트 종료! 걷어차이면서도 버틴 시간: {score} 프레임")

env.close()

😈 극한의 방해꾼 테스트를 시작합니다! 화면을 주목하세요!

--- 1번째 가혹한 테스트 시작! ---
💥 퍽! ⬅️ 왼쪽으로 걷어찼습니다! (가해진 힘: -0.09)
💥 퍽! 오른쪽 ➡️으로 걷어찼습니다! (가해진 힘: 1.22)
💥 퍽! 오른쪽 ➡️으로 걷어찼습니다! (가해진 힘: 1.91)
💥 퍽! 오른쪽 ➡️으로 걷어찼습니다! (가해진 힘: 1.24)
💥 퍽! ⬅️ 왼쪽으로 걷어찼습니다! (가해진 힘: -0.22)
💥 퍽! 오른쪽 ➡️으로 걷어찼습니다! (가해진 힘: 0.70)
💥 퍽! ⬅️ 왼쪽으로 걷어찼습니다! (가해진 힘: -0.62)
💥 퍽! 오른쪽 ➡️으로 걷어찼습니다! (가해진 힘: 1.71)
테스트 종료! 걷어차이면서도 버틴 시간: 1000.0 프레임

--- 2번째 가혹한 테스트 시작! ---
💥 퍽! 오른쪽 ➡️으로 걷어찼습니다! (가해진 힘: 0.63)
💥 퍽! 오른쪽 ➡️으로 걷어찼습니다! (가해진 힘: 1.48)
💥 퍽! 오른쪽 ➡️으로 걷어찼습니다! (가해진 힘: 1.98)
테스트 종료! 걷어차이면서도 버틴 시간: 226.0 프레임

--- 3번째 가혹한 테스트 시작! ---
💥 퍽! ⬅️ 왼쪽으로 걷어찼습니다! (가해진 힘: -0.04)
💥 퍽! ⬅️ 왼쪽으로 걷어찼습니다! (가해진 힘: -1.47)
💥 퍽! ⬅️ 왼쪽으로 걷어찼습니다! (가해진 힘: -1.51)
💥 퍽! ⬅️ 왼쪽으로 걷어찼습니다! (가해진 힘: -1.95)
테스트 종료! 걷어차이면서도 버틴 시간: 735.0 프레임

--- 4번째 가혹한 테스트 시작! ---
💥 퍽! 오른쪽 ➡️으로 걷어찼습니다! (가해진 힘: 0.06)
💥 퍽! 오른쪽 ➡️으로 걷어찼습니다! (가해진 힘: 0.18)
💥 퍽! 오른쪽 ➡️으로 걷어찼습니다! (가해진 힘: 0.33)
💥 퍽! 오른쪽 ➡️으로 걷어찼습니다! (가해진 힘: 0.20)
💥 퍽! 오른쪽 ➡️으로 걷어찼습니다! (가해진 힘: 1.59)
💥 퍽! ⬅️ 왼쪽으로 걷어찼습니다! (가해진 힘: -0.65)
💥 퍽! 오른쪽 ➡️